In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("lucid-parsec-464412-t8-75d0c2e2a5fa.json", scope)
client = gspread.authorize(creds)


sheet = client.open("Spotify History").sheet1


import csv
with open("Spotify_History.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(sheet.row_values(1))  # Write header
    writer.writerows(sheet.get_all_values()[1:])  # Write data

print("Spotify History Sheet downloaded and saved as 'Spotify_History.csv'.")

Spotify History Sheet downloaded and saved as 'Spotify_History.csv'.


In [ ]:

sheet = client.open("Recent Spotify Plays").sheet1

import csv
with open("Recent_Spotify_History.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(sheet.row_values(1)) 
    writer.writerows(sheet.get_all_values()[1:]) 

print("Spotify Historydownloaded'.")

Spotify Historydownloaded'.


In [ ]:
import pandas as pd

df = pd.read_csv("Recent_Spotify_History.csv")

if "URI" in df.columns:
    df = df.drop(columns=["URI"])

df.to_csv("Recent_Spotify_History.csv", index=False)

print("URI column removed from 'Recent_Spotify_History.csv'.")

URI column removed from 'Recent_Spotify_History.csv'.


In [ ]:
import pandas as pd

history_df = pd.read_csv("Spotify_History2014-Aug2025.csv")
recent_df = pd.read_csv("Recent_Spotify_History.csv")

combined_df = pd.concat([history_df, recent_df], ignore_index=True)

num_duplicates = combined_df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

combined_df = combined_df.drop_duplicates()

print("Recent history appended, duplicates removed, and saved to 'Spotify_History2014-Aug2025.csv'.")

Number of duplicate rows: 171
Recent history appended, duplicates removed, and saved to 'Spotify_History2014-Aug2025.csv'.


In [ ]:
combined_df.to_csv("Spotify_History2014-Sept2025.csv", index=False)